In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib, pickle
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import xgboost as xgb

In [2]:
# Load the cleaned dataset
data = pd.read_csv("/content/cleaned_air_quality (1).csv")
print("Loaded dataset:", data.shape)
print("Columns:", list(data.columns))
print(data.head())

# We will predict PM2.5 levels per city
target_col = "PM2.5"


Loaded dataset: (7957, 11)
Columns: ['City', 'Country', 'PM2.5', 'PM10', 'NO2', 'SO2', 'CO', 'O3', 'Temperature', 'Humidity', 'Wind Speed']
             City   Country   PM2.5    PM10    NO2    SO2    CO      O3  \
0         Bangkok  Thailand   86.57   25.19  99.88  30.63  4.46   36.29   
1        Istanbul    Turkey   50.63   97.39  48.14   8.71  3.40  144.16   
2  Rio de Janeiro    Brazil  130.21   57.22  98.51   9.92  0.12  179.31   
3          Mumbai     India  119.70  130.52  10.96  33.03  7.74   38.65   
4           Paris    France   55.20   36.62  76.85  21.85  2.00   67.09   

   Temperature  Humidity  Wind Speed  
0        17.67     59.35       13.76  
1         3.46     67.51        6.36  
2        25.29     29.30       12.87  
3        23.15     99.97        7.71  
4        16.02     90.28       14.16  


In [3]:
# Function to calculate MAE and RMSE
def evaluate_model(actual, predicted):
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    return mae, rmse

In [6]:
def train_models_for_city(city_data, city_name):
    print(f"\nTraining models for city: {city_name}")
    series = city_data[target_col].astype(float).reset_index(drop=True)

    # Original 'Date' column is unusable)
    city_data = city_data.copy()
    city_data["ds"] = pd.date_range(start="2020-01-01", periods=len(series), freq="D")
    city_data["y"] = series

    # Train/test split
    split = int(len(series) * 0.8)
    train = series[:split]
    test = series[split:]
    actual = test.values

    results = {}

    # ARIMA
    try:
        arima_model = ARIMA(train, order=(2, 1, 2))
        arima_fit = arima_model.fit()
        pred_arima = arima_fit.forecast(steps=len(test))
        results["ARIMA"] = evaluate_model(actual, pred_arima)
    except Exception as e:
        print("ARIMA failed:", e)
        results["ARIMA"] = (np.nan, np.nan)

    # Prophet
    try:
        train_p = city_data.iloc[:split][["ds", "y"]]
        test_p = city_data.iloc[split:][["ds", "y"]]

        prophet_model = Prophet()
        prophet_model.fit(train_p)

        future = prophet_model.make_future_dataframe(periods=len(test_p), freq="D")
        forecast = prophet_model.predict(future)
        pred_prophet = forecast["yhat"].iloc[-len(test_p):].values
        results["Prophet"] = evaluate_model(actual, pred_prophet)
    except Exception as e:
        print("Prophet failed:", e)
        results["Prophet"] = (np.nan, np.nan)

    # LSTM
    try:
        scaler = MinMaxScaler()
        scaled = scaler.fit_transform(series.values.reshape(-1, 1))

        def create_dataset(data, lag=5):
            X, y = [], []
            for i in range(lag, len(data)):
                X.append(data[i - lag:i, 0])
                y.append(data[i, 0])
            return np.array(X), np.array(y)

        X, y = create_dataset(scaled)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        split_lstm = int(len(X) * 0.8)
        X_train, X_test = X[:split_lstm], X[split_lstm:]
        y_train, y_test = y[:split_lstm], y[split_lstm:]

        lstm_model = Sequential([
            LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)),
            Dense(1)
        ])
        lstm_model.compile(optimizer='adam', loss='mse')
        lstm_model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)

        y_pred = lstm_model.predict(X_test)
        y_pred_inv = scaler.inverse_transform(y_pred)
        y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

        results["LSTM"] = evaluate_model(y_test_inv.flatten(), y_pred_inv.flatten())
    except Exception as e:
        print("LSTM failed:", e)
        results["LSTM"] = (np.nan, np.nan)

    # XGBoost
    try:
        df_lag = pd.DataFrame(series)
        for lag in range(1, 4):
            df_lag[f"lag_{lag}"] = df_lag[target_col].shift(lag)
        df_lag.dropna(inplace=True)

        X = df_lag[[f"lag_{i}" for i in range(1, 4)]]
        y = df_lag[target_col]
        split_xgb = int(len(X) * 0.8)
        X_train, X_test = X.iloc[:split_xgb], X.iloc[split_xgb:]
        y_train, y_test = y.iloc[:split_xgb], y.iloc[split_xgb:]

        xgb_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=150)
        xgb_model.fit(X_train, y_train)
        pred_xgb = xgb_model.predict(X_test)

        results["XGBoost"] = evaluate_model(y_test, pred_xgb)
    except Exception as e:
        print("XGBoost failed:", e)
        results["XGBoost"] = (np.nan, np.nan)

    # Compare results
    result_df = pd.DataFrame(results, index=["MAE", "RMSE"]).T
    print("\nModel Performance Summary:")
    print(result_df)

    # Pick best model (lowest RMSE)
    best_model = min(results, key=lambda k: results[k][1] if not np.isnan(results[k][1]) else np.inf)
    print(f"Best model for {city_name}: {best_model}")

    # Save best model
    if best_model == "ARIMA":
        joblib.dump(arima_fit, f"{city_name}_best_model.pkl")
    elif best_model == "Prophet":
        pickle.dump(prophet_model, open(f"{city_name}_best_model.pkl", "wb"))
    elif best_model == "LSTM":
        lstm_model.save(f"{city_name}_best_model.h5")
    elif best_model == "XGBoost":
        joblib.dump(xgb_model, f"{city_name}_best_model.pkl")

    print(f"Saved best model for {city_name}\n")

    return result_df


In [11]:
cities_to_run = ["Mumbai", "Paris", "Tokyo"]

for city in cities_to_run:
    city_data = data[data["City"] == city]
    if not city_data.empty:
        train_models_for_city(city_data, city)
    else:
        print(f"No data found for {city}")



Training models for city: Mumbai


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 18.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpucjx0wa6/wuas5gvh.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpucjx0wa6/pf10pnzd.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=85159', 'data', 'file=/tmp/tmpucjx0wa6/wuas5gvh.json', 'init=/tmp/tmpucjx0wa6/pf10pnzd.json', 'output', 'file=/tmp/tmpucjx0wa6/prophet_modelhi5algxy/prophet_model-202

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step

Model Performance Summary:
                MAE        RMSE
ARIMA     46.573521   50.333225
Prophet  835.064751  838.795288
LSTM      46.941708   50.944909
XGBoost   47.601177   55.346927
Best model for Mumbai: ARIMA
Saved best model for Mumbai


Training models for city: Paris


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 17.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpucjx0wa6/7yhobnwb.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpucjx0wa6/i_d4xln8.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/li

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step



Model Performance Summary:
                MAE        RMSE
ARIMA     32.773664   37.450219
Prophet  536.390687  538.720398
LSTM      31.914418   36.770481
XGBoost   37.904328   46.733681
Best model for Paris: LSTM
Saved best model for Paris


Training models for city: Tokyo


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 10.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpucjx0wa6/e5e6fhse.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpucjx0wa6/pppfpqzk.json
DEBUG:cmdstanpy:

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step

Model Performance Summary:
                 MAE         RMSE
ARIMA      33.058993    38.363569
Prophet  1061.630974  1064.432029
LSTM       32.242974    38.386472
XGBoost    35.204938    42.584474
Best model for Tokyo: ARIMA
Saved best model for Tokyo

